In [1]:
import pandas as pd
import requests

url = "https://cdec.water.ca.gov/dynamicapp/req/JSONDataServlet?Stations=ADM%2CBKL%2CBMW%2CBNK%2CCDP%2CDSS%2CFOR%2CGOL%2CGRZ%2CHMB%2CIDC%2CIDP%2CINN%2CLLP%2CMED%2CNLS%2CPLP%2CRRM%2CRTL%2CSHM%2CSLT%2CSNM%2CSQV%2CTK2%2C+ALP%2CBLC%2CBLK%2CBLS%2CBSK%2CCAP%2CCSL%2CDAN%2CDDM%2CEBB%2CEP5%2CFDC%2CFLL%2CGIN%2CGKS%2CGNL%2CHGM%2CHOR%2CHRS%2CHVN%2CHYS%2CLBD%2CLVM%2CLVT%2CMDW%2CMNT%2CMRL%2CMSK%2CPSN%2CRBB%2CRBP%2CRP2%2CSDW%2CSIL%2CSLI%2CSPS%2CSPT%2CSTR%2CTCC%2CTNY%2CTUM%2CVRG%2CWC3%2CWHW%2CBCH%2CBGP%2CCHM%2CCHP%2CCRL%2CCSV%2CCWD%2CDPO%2CGNF%2CGRM%2CGRV%2CKSP%2CMTM%2CPSC%2CPSR%2CRCK%2CSLK%2CSTL%2CSWM%2CTMR%2CTUN%2CUTY%2CVLC%2CWTM&SensorNums=18&dur_code=D&Start=2013-01-01&End=2023-04-13"
response = requests.get(url).json()

In [2]:
station_list = []
date_list = []
value_list = []

for station in response:
    if "obsDate" in station:
        station_id = station['stationId']
        date = pd.to_datetime(station['obsDate'])
        value = station['value']
        station_list.append(station_id)
        date_list.append(date)
        value_list.append(value)

snow_df = pd.DataFrame({
    "Date" : date_list,
    "Station ID" : station_list,
    "Snow Depth (in)" : value_list,
})

snow_df["Month"] = snow_df["Date"].dt.month
snow_df["Year"] = snow_df["Date"].dt.year

array(['ADM', 'BNK', 'IDC', 'IDP', 'INN', 'LLP', 'MED', 'NLS', 'RRM',
       'SHM', 'SLT', 'SNM', 'SQV', 'TK2', 'BLC', 'BLK', 'BLS', 'BSK',
       'CAP', 'DAN', 'DDM', 'EBB', 'EP5', 'FDC', 'FLL', 'GIN', 'GKS',
       'GNL', 'HGM', 'HOR', 'HRS', 'HVN', 'HYS', 'LVM', 'LVT', 'MDW',
       'MNT', 'MRL', 'MSK', 'PSN', 'RP2', 'SDW', 'SIL', 'SLI', 'SPS',
       'SPT', 'STR', 'TCC', 'TNY', 'TUM', 'VRG', 'WC3', 'WHW', 'BCH',
       'BGP', 'CHM', 'CHP', 'CRL', 'CSV', 'CWD', 'DPO', 'GRM', 'GRV',
       'KSP', 'MTM', 'PSC', 'PSR', 'RCK', 'SLK', 'STL', 'SWM', 'TMR',
       'TUN', 'UTY', 'VLC', 'WTM'], dtype=object)

In [11]:
winter_bin = [0,4,10,12]
labels = ["Winter_End", "Summer", "Winter_Start"]

snow_binned = snow_df.copy()
snow_binned["Season"] = pd.cut(x = snow_binned["Month"], bins =  winter_bin, labels = labels)

snow_season = snow_binned.sort_values(["Year", "Month"])

season_list = []
working_year = 2014

for row in snow_season.iterrows():
    row_year = row[1][4]
    row_season = row[1][5]

    if (row_season == "Winter_End") & (row_year == working_year):
        season_list.append(f"Winter {working_year - 1} {working_year}")
    elif (row_season == "Winter_Start") & (row_year == (working_year - 1)):
        season_list.append(f"Winter {working_year - 1} {working_year}")
    elif (row_season == "Summer"):
        season_list.append(f"Summer {working_year - 1} {working_year}")
    else:
        season_list.append(f"na")

    if (row_year == (working_year + 1)):
        working_year = working_year + 1

snow_season["Season"] = season_list

In [20]:
winter_only_mask = snow_season["Season"].str.contains("Winter")
snow_season_winter_only = snow_season.loc[winter_only_mask]
snow_season_grouped = snow_season_winter_only.groupby(["Season"])
snow_season_grouped.head()

,Date,Station ID,Snow Depth (in),Month,Year,Season
304,2013-11-01,ADM,0,11,2013,Winter 2013 2014
305,2013-11-02,ADM,0,11,2013,Winter 2013 2014
306,2013-11-03,ADM,0,11,2013,Winter 2013 2014
307,2013-11-04,ADM,0,11,2013,Winter 2013 2014
308,2013-11-05,ADM,0,11,2013,Winter 2013 2014
727,2015-01-02,ADM,6,1,2015,Winter 2014 2015
728,2015-01-03,ADM,7,1,2015,Winter 2014 2015
729,2015-01-04,ADM,6,1,2015,Winter 2014 2015
730,2015-01-05,ADM,6,1,2015,Winter 2014 2015
731,2015-01-06,ADM,6,1,2015,Winter 2014 2015
